<a href="https://colab.research.google.com/github/DarkNightSoldier/DTApp/blob/main/ProyectoBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tabula-py
!pip install pandasql
!pip install python-docx
!wget https://raw.githubusercontent.com/DarkNightSoldier/DTApp/main/DatosApp.db

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 9.2 MB/s eta 0:00:00


In [ ]:
import tabula
import pandas as pd
import re
import sqlite3
from pandasql import sqldf

import warnings
from datetime import datetime
import pandas as pd
from docx import Document
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.shared import Mm  # Importar Mm para definir tamaños en milímetros

warnings.simplefilter(action='ignore', category=Warning)

# Conexión a la base de datos SQLite
Apuntador = sqlite3.connect('DatosApp.db')
cursor = Apuntador.cursor()

def formatear_tupla(array):
  valores = tuple(array)
  if len(valores) == 1:
      valores_str = f"({valores[0]})"  # Sin coma si hay solo un valor
  else:
      valores_str = str(tuple(valores))  # Convierte la lista en tupla (válida en SQL)
  return valores_str

import pandas as pd
import re
import tabula

import pandas as pd
import re
import tabula

def Obtener_Informacion_PDF(pdf_path):
    # Obtener todas las tablas presentes en el PDF
    tablas = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)

    # Capturar tabla con información básica del estudiante
    info_estudiante = pd.DataFrame(tablas[1].iloc[:, :2].values).dropna()
    for i in range(len(tablas)):
        if len(tablas[i][tablas[i].iloc[:, 0] == "Porcentaje de avance"]) > 0:
            info_estudiante = pd.concat([info_estudiante, pd.DataFrame(tablas[i].values).iloc[:, :2].dropna()])
        if "P.A.P.A" in tablas[i].columns:
            PAPA = tablas[i]["P.A.P.A"][0].replace(',', '.')
            info_estudiante = pd.concat([info_estudiante, pd.DataFrame([["P.A.P.A:", PAPA]])])
        if len(tablas[i][tablas[i].iloc[:, 0] == "Créditos de estudio doble titulación"]) > 0:
            nueva_info = pd.DataFrame(tablas[i].iloc[:, :2].values)
            if nueva_info.loc[nueva_info[0] == "Créditos de estudio doble titulación"][1].isna().values[0]:
                nueva_info = pd.DataFrame(tablas[i].iloc[:, [0, 2]].values)
            info_estudiante = pd.concat([info_estudiante, nueva_info.dropna()])

    # Construir la tabla con la información de asignaturas
    asignaturas_estudiante = None
    for i in range(len(tablas)):
        if len(tablas[i][tablas[i].iloc[:, 0] == "Asignatura"]) > 0 or "Asignatura" in tablas[i].columns:
            if asignaturas_estudiante is None:
                asignaturas_estudiante = pd.DataFrame(tablas[i].values)
            else:
                asignaturas_estudiante = pd.concat([asignaturas_estudiante, pd.DataFrame(tablas[i].values)])

    # Unir filas fragmentadas
    filas_completas = []
    fila_actual = None

    for _, fila in asignaturas_estudiante.iterrows():
        if pd.notna(fila[0]):  # Si la fila tiene una asignatura, es una nueva fila válida
            if fila_actual is not None:
                filas_completas.append(fila_actual)
            fila_actual = fila.copy()
        elif fila_actual is not None:
            for col in asignaturas_estudiante.columns:
                if pd.isna(fila_actual[col]) and pd.notna(fila[col]):
                    fila_actual[col] = fila[col]

    if fila_actual is not None:
        filas_completas.append(fila_actual)

    asignaturas_estudiante = pd.DataFrame(filas_completas)
    asignaturas_estudiante.reset_index(drop=True, inplace=True)

    # Eliminar encabezados repetidos y asignar columnas
    asignaturas_estudiante.dropna(subset=[0], inplace=True)
    asignaturas_estudiante.columns = asignaturas_estudiante.iloc[0]
    asignaturas_estudiante = asignaturas_estudiante.drop(asignaturas_estudiante.index[0])

    # Periodo
    periodo_inicio = info_estudiante.loc[info_estudiante[0] == "Periodo de inicio:"][1].values[0]
    asignaturas_estudiante['Periodo'] = None
    periodo_actual = periodo_inicio

    for i, row in asignaturas_estudiante.iterrows():
        asignatura_str = str(row['Asignatura'])
        if "PERIODO" in asignatura_str:
            match = re.search(r'(\d{4}-\dS)', asignatura_str)
            if match:
                periodo_actual = match.group(1)
        else:
            asignaturas_estudiante.at[i, 'Periodo'] = periodo_actual

    asignaturas_estudiante = asignaturas_estudiante[~asignaturas_estudiante['Asignatura'].str.contains("PERIODO", na=False)]

    # Formatear los datos
    asignaturas_estudiante["Cod_Asignatura"] = asignaturas_estudiante['Asignatura'].str.extract(r'\((\d+)\)')
    asignaturas_estudiante['Cod_Asignatura'] = asignaturas_estudiante['Cod_Asignatura'].fillna(method='bfill')
    asignaturas_estudiante['Nom_Asignatura'] = (
        asignaturas_estudiante['Asignatura']
        .str.replace(r'\([^)]*\)', '', regex=True)
        .str.strip()
    )

    # Convertir tipos de datos
    asignaturas_estudiante["Creditos"] = pd.to_numeric(asignaturas_estudiante["Créditos"], errors="coerce").fillna(0).astype(int)
    asignaturas_estudiante["Calificacion"] = (
        asignaturas_estudiante["Calificación"].str.extract(r'(\d+(,\d+)?)')[0]
        .str.replace(",", ".", regex=False)
        .astype(float)
    )

    # Filtrar asignaturas válidas
    asignaturas_estudiante = asignaturas_estudiante[asignaturas_estudiante['Calificacion'].notna()]
    asignaturas_estudiante = asignaturas_estudiante[asignaturas_estudiante['Anulada'] == 'NO']
    asignaturas_estudiante = asignaturas_estudiante[asignaturas_estudiante['Calificacion'] >= 3]
    asignaturas_estudiante = asignaturas_estudiante[asignaturas_estudiante["Tipología"] != "Nivelación (E)"]
    # Trabajo de grado no es equivalible ni convalidable
    asignaturas_estudiante = asignaturas_estudiante[~asignaturas_estudiante["Tipología"].str.contains(r"\(P\)", na=False)]

    # Seleccionar columnas finales
    asignaturas_estudiante = asignaturas_estudiante[['Periodo', 'Cod_Asignatura', 'Nom_Asignatura', 'Creditos', 'Calificacion']]

    return info_estudiante, asignaturas_estudiante


# Queremos obtener una lista de EQUIVALENCIAS CANDIDATAS

def Generar_Lista_Candidatas(asignaturas_estudiante):
    # Query para obtener las asignaturas que cursó y que pertenecen al programa
    # Esto es Equivalencias con igual código
    lista_asignaturas_cursadas = tuple(asignaturas_estudiante["Cod_Asignatura"])
    query_1 = f"""
        SELECT
            Cod_Asignatura_CC AS Código,
            Nom_Agrupacion AS Agrupación,
            Tipo
        FROM Asignaturas_CC
        WHERE Cod_Asignatura_CC IN {formatear_tupla(lista_asignaturas_cursadas)}"""
    asignaturas_iguales = pd.read_sql_query(query_1, Apuntador)

    # Join para completar la información de las asignaturas
    query_2 = """
        SELECT
            asignaturas_estudiante.Periodo AS "Periodo Académico",
            asignaturas_estudiante.Cod_Asignatura AS "Código",
            asignaturas_estudiante.Nom_Asignatura AS "Asignatura",
            asignaturas_iguales.Código AS "Código_CC",
            asignaturas_estudiante.Nom_Asignatura AS "Asignatura_CC",
            asignaturas_iguales.Agrupación AS "Agrupación",
            asignaturas_estudiante.Calificacion AS "Nota",
            asignaturas_iguales.Tipo AS "Tipo",
            asignaturas_estudiante.Creditos AS "Créditos"
        FROM asignaturas_estudiante
        JOIN asignaturas_iguales
        ON asignaturas_iguales.Código = asignaturas_estudiante.Cod_Asignatura;
    """

    equivalencias_candidatas = pd.DataFrame(sqldf(query_2))


    # Equivalencias con diferente código
    # Obtener la equivalencia desde la base de datos
    query_3 = f"""
        SELECT
            Cod_Asignatura AS Código,
            Cod_Asignatura_CC AS 'Código_CC'
        FROM Equivalencias
        WHERE Cod_Asignatura IN {formatear_tupla(lista_asignaturas_cursadas)}"""
    asignaturas_equivalentes = pd.read_sql_query(query_3, Apuntador)

    # Join para obtener la agrupación y tipo de las asignaturas equivalentes
    query_4 = f"""
        SELECT
            Cod_Asignatura_CC AS 'Código_CC',
            Nom_Agrupacion AS Agrupación,
            Tipo
        FROM Asignaturas_CC
        WHERE Cod_Asignatura_CC IN {formatear_tupla(asignaturas_equivalentes["Código_CC"])}"""
    info_equivalentes = pd.read_sql_query(query_4, Apuntador)

    # Join para obtener el número de créditos y nombre de las asignaturas equivalentes
    query_5 = f"""
        SELECT
            Cod_Asignatura AS Código_CC,
            Nom_Asignatura AS Asignatura_CC,
            Creditos
        FROM Asignaturas_Info
        WHERE Asignaturas_Info IN {formatear_tupla(asignaturas_equivalentes["Código_CC"])}"""
    info_equivalentes_2 = pd.read_sql_query(query_5, Apuntador)

    # Join para completar la información de las asignaturas
    query_6 = """
    SELECT
        e.Periodo AS "Periodo Académico",
        e.Cod_Asignatura AS "Código",
        e.Nom_Asignatura AS "Asignatura",
        a.Código_CC AS "Código_CC",
        ie2.Asignatura_CC AS "Asignatura_CC",
        ie.Agrupación AS "Agrupación",
        ie2.Creditos AS "Créditos",
        ie.Tipo AS "Tipo"
    FROM asignaturas_estudiante AS e
    JOIN asignaturas_equivalentes AS a
    ON e.Cod_Asignatura = a.Código
    JOIN info_equivalentes AS ie
    ON a.Código_CC = ie.Código_CC
    JOIN info_equivalentes_2 AS ie2
    ON ie.Código_CC = ie2.Código_CC;
    """
    # Concatenar las equivalencias de asignaturas iguales con las de diferentes
    equivalencias_candidatas = pd.concat([equivalencias_candidatas,pd.DataFrame(sqldf(query_6))],axis=0)
    return equivalencias_candidatas

def Generar_Estudio(info_estudiante,lista_candidatas):
    # Consulta de agrupaciones
    query_7 = 'SELECT * FROM Agrupaciones_CC'
    agrupaciones = pd.read_sql_query(query_7, Apuntador)
    agrupaciones

    # Cursadas en Fundamentación (Por el momento únicamente las obligatorias)
    cursadas_fundamentacion = lista_candidatas[lista_candidatas["Tipo"] == "B"]

    # Pendientes en Fundamentación - Obligatoria B
    query_8 = f"""
    SELECT
        Cod_Asignatura_CC AS 'Código',
        Nom_Agrupacion AS 'Agrupación'
    FROM Asignaturas_CC
    WHERE Cod_Asignatura_CC NOT IN {formatear_tupla(cursadas_fundamentacion["Código_CC"])} AND Tipo = 'B'
    """
    pendientes_b = pd.read_sql_query(query_8, Apuntador)

    query_9 = f"""
    SELECT
        Cod_Asignatura as 'Código',
        Nom_Asignatura as 'Asignatura',
        Creditos as 'Créditos'
    FROM Asignaturas_Info
    WHERE Cod_Asignatura IN {formatear_tupla(pendientes_b["Código"])}
    """
    pendientes_b_info = pd.read_sql_query(query_9, Apuntador)
    pendientes_b_info

    query_10 = f"""
    SELECT
        p.Agrupación AS 'Agrupación',
        i.Código as 'Código',
        i.Asignatura as 'Asignatura',
        i.Créditos as 'Créditos'
    FROM pendientes_b_info as i
    JOIN pendientes_b as p
    ON i.Código = p.Código
    """
    tabla_pendientes_b = pd.DataFrame(sqldf(query_10)).sort_values(by="Agrupación")

    # Calcular la suma de créditos por agrupación
    suma_creditos = tabla_pendientes_b.groupby("Agrupación")["Créditos"].sum().reset_index()
    suma_creditos.rename(columns={"Créditos": "Créditos pendientes por cursar de la agrupación"}, inplace=True)

    # Unir esta información al DataFrame original
    tabla_pendientes_b = tabla_pendientes_b.merge(suma_creditos, on="Agrupación", how="left")

    # Pendientes en Disciplinar - Obligatoria C y Trabajo de Grado P
    # Cursadas en Fundamentación (Por el momento únicamente las obligatorias)
    cursadas_disciplinar = lista_candidatas[lista_candidatas["Tipo"] == "C"]

    # Agrupación de fundamentación
    lista_agrup_fund = agrupaciones[agrupaciones["Tipologia"]=="Disciplinar"]["Nom_Agrupacion"]

    query_11 = f"""
    SELECT
        Cod_Asignatura_CC AS 'Código',
        Nom_Agrupacion AS 'Agrupación'
    FROM Asignaturas_CC
    WHERE Cod_Asignatura_CC NOT IN {formatear_tupla(cursadas_disciplinar["Código_CC"])} AND Tipo = 'C'
    """
    pendientes_c = pd.read_sql_query(query_11, Apuntador)

    query_12 = f"""
    SELECT
        Cod_Asignatura as 'Código',
        Nom_Asignatura as 'Asignatura',
        Creditos as 'Créditos'
    FROM Asignaturas_Info
    WHERE Cod_Asignatura IN {formatear_tupla(pendientes_c["Código"])}
    """
    pendientes_c_info = pd.read_sql_query(query_12, Apuntador)
    pendientes_c_info

    query_13 = f"""
    SELECT
        p.Agrupación AS 'Agrupación',
        i.Código as 'Código',
        i.Asignatura as 'Asignatura',
        i.Créditos as 'Créditos'
    FROM pendientes_c_info as i
    JOIN pendientes_c as p
    ON i.Código = p.Código
    """
    tabla_pendientes_c = pd.DataFrame(sqldf(query_13)).sort_values(by="Agrupación")

    # Calcular la suma de créditos por agrupación
    suma_creditos = tabla_pendientes_c.groupby("Agrupación")["Créditos"].sum().reset_index()
    suma_creditos.rename(columns={"Créditos": "Créditos pendientes por cursar de la agrupación"}, inplace=True)

    # Unir esta información al DataFrame original
    tabla_pendientes_c = tabla_pendientes_c.merge(suma_creditos, on="Agrupación", how="left")


    # Pendientes en Disciplinar - Obligatoria P Trabajo de Grado
    query_14 = f"""
    SELECT
        a.Nom_Agrupacion AS 'Agrupación',
        a.Cod_Asignatura_CC AS 'Código',
        i.Nom_Asignatura AS 'Asignatura',
        i.Creditos AS 'Créditos',
        agrup.Cant_Optativos AS 'Créditos pendientes por cursar de la agrupación'
    FROM Asignaturas_CC AS a
    JOIN Asignaturas_Info AS i
    ON i.Cod_Asignatura = a.Cod_Asignatura_CC
    JOIN Agrupaciones_CC AS agrup
    ON agrup.Nom_Agrupacion = a.Nom_Agrupacion
    WHERE a.Nom_Agrupacion = 'TRABAJO DE GRADO'
    """
    tabla_pendientes_disciplinar = pd.concat([tabla_pendientes_c,pd.read_sql_query(query_14, Apuntador)]).reset_index(drop=True)

    # Contador de créditos Optativas O y T
    creditos_contados = pd.DataFrame(columns=["Agrupación","Tipo","Créditos aprobados","Créditos excedentes"])

    # Seleccionar en Fundamentación Optativa - O las asignaturas a equivaler
    # Agrupación de fundamentación
    lista_agrup_fund = agrupaciones[agrupaciones["Tipologia"]=="Fundamentación"]["Nom_Agrupacion"]

    # Seleccionar las asignaturas con mayor nota hasta completar la agrupación
    for agrupacion in lista_agrup_fund:
        cantidad_creditos_o = agrupaciones[agrupaciones["Nom_Agrupacion"]==agrupacion]["Cant_Optativos"].values[0]
        asignaturas = lista_candidatas[(lista_candidatas["Agrupación"] == agrupacion) & (lista_candidatas["Tipo"] == "O")].sort_values(by="Nota", ascending=False).reset_index(drop=True)
        suma_creditos = 0
        i = 0
        while suma_creditos < cantidad_creditos_o and i<len(asignaturas):
            # Seleccionar el primer registro - el que tiene mejor nota
            asignatura = asignaturas.iloc[0]

            # Agregarlo a cursadas_fundamentacion
            cursadas_fundamentacion = pd.concat([cursadas_fundamentacion, pd.DataFrame([asignatura])], ignore_index=True)
            suma_creditos = suma_creditos + asignatura["Créditos"]

            # Eliminar el primer registro de tabla
            asignaturas = asignaturas.iloc[1:].reset_index(drop=True)

            # Incrementar el contador
            i += 1
        creditos_contados = pd.concat([creditos_contados, pd.DataFrame([[agrupacion,"Fundamentación", suma_creditos, max(suma_creditos-cantidad_creditos_o,0)]], columns=["Agrupación", "Tipo", "Créditos aprobados","Créditos excedentes"])], ignore_index=True)


    # Seleccionar en Disciplinar - T las asignaturas a equivaler
    # Agrupación de Disciplinar
    lista_agrup_disciplinar = agrupaciones[agrupaciones["Tipologia"]=="Disciplinar"]["Nom_Agrupacion"]

    # Seleccionar las asignaturas con mayor nota hasta completar la agrupación
    for agrupacion in lista_agrup_disciplinar:
        cantidad_creditos_t = agrupaciones[agrupaciones["Nom_Agrupacion"]==agrupacion]["Cant_Optativos"].values[0]
        asignaturas = lista_candidatas[(lista_candidatas["Agrupación"] == agrupacion) & (lista_candidatas["Tipo"] == "T")].sort_values(by="Nota", ascending=False).reset_index(drop=True)
        suma_creditos = 0
        i = 0
        while suma_creditos < cantidad_creditos_t and i<len(asignaturas):
            # Seleccionar el primer registro - el que tiene mejor nota
            asignatura = asignaturas.iloc[0]

            # Agregarlo a cursadas_fundamentacion
            cursadas_disciplinar = pd.concat([cursadas_disciplinar, pd.DataFrame([asignatura])], ignore_index=True)
            suma_creditos = suma_creditos + int(asignatura["Créditos"])

            # Eliminar el primer registro de tabla
            asignaturas = asignaturas.iloc[1:].reset_index(drop=True)

            # Incrementar el contador
            i += 1
        creditos_contados = pd.concat([creditos_contados, pd.DataFrame([[agrupacion, "Disciplinar", suma_creditos, max(suma_creditos-cantidad_creditos_t,0)]], columns=["Agrupación","Tipo","Créditos aprobados","Créditos excedentes"])], ignore_index=True)


    # Visualizar pendientes en Fundamentación Optativa O
    # Pendientes en Fundamentación - Optativa O
    pendientes_o = agrupaciones[agrupaciones["Tipologia"]=="Fundamentación"][["Nom_Agrupacion", "Cant_Optativos"]]
    pendientes_o.rename(columns={"Nom_Agrupacion": "Agrupación", "Cant_Optativos": "Créditos exigidos en la agrupación"}, inplace=True)
    pendientes_o["Créditos pendientes por cursar de la agrupación"] = (
        pendientes_o["Créditos exigidos en la agrupación"] -
        creditos_contados[creditos_contados["Agrupación"].isin(pendientes_o["Agrupación"])]["Créditos aprobados"].values
    ).clip(lower=0)

    # Visualizar pendientes en Disciplinar Optativa T
    # Pendientes en Disciplinar - Optativa T
    pendientes_t = agrupaciones[agrupaciones["Tipologia"]=="Disciplinar"][["Nom_Agrupacion", "Cant_Optativos"]]
    pendientes_t.rename(columns={"Nom_Agrupacion": "Agrupación", "Cant_Optativos": "Créditos exigidos en la agrupación"}, inplace=True)
    pendientes_t["Créditos pendientes por cursar de la agrupación"] = (
        pendientes_t["Créditos exigidos en la agrupación"] -
        creditos_contados[creditos_contados["Agrupación"].isin(pendientes_t["Agrupación"])]["Créditos aprobados"].values
    ).clip(lower=0)

    ### CORREGIR NO ES QUE NO ESTÉ EN CURSADAS SI NO QUE NO ESTÉ EN LA LISTA DE ASIGNATURAS DE LA CARRERA ###
    # Seleccionar en Libre Elección - L las asignaturas a equivaler
    candidatas_libre = asignaturas_estudiante[~(asignaturas_estudiante["Cod_Asignatura"].isin(lista_candidatas["Código"]))]
    # Ordenar por calificación
    candidatas_libre = candidatas_libre.sort_values(by="Calificacion", ascending=False).reset_index(drop=True)

    # Seleccionar en Libre Elección - L las asignaturas a equivaler
    creditos_libre = agrupaciones[agrupaciones["Tipologia"]=="Libre Elección"]["Cant_Optativos"].values[0]
    cursadas_libre = pd.DataFrame(columns=candidatas_libre.columns)
    suma_creditos = 0
    i = 0
    while suma_creditos<creditos_libre and i<len(candidatas_libre):
        # Seleccionar el primer registro - el que tiene mejor nota
        asignatura = candidatas_libre.iloc[0]

        # Agregarlo a cursadas_libre
        cursadas_libre = pd.concat([cursadas_libre, pd.DataFrame([asignatura])], ignore_index=True)
        suma_creditos = suma_creditos + int(asignatura["Creditos"])

        # Eliminar el primer registro de tabla
        candidatas_libre = candidatas_libre.iloc[1:].reset_index(drop=True)

        # Incrementar el contador
        i += 1

    #Guardar dato de los excedentes
    creditos_contados = pd.concat([creditos_contados, pd.DataFrame([["LIBRE ELECCIÓN", "Libre Elección", suma_creditos, max(suma_creditos-creditos_libre,0)]], columns=["Agrupación","Tipo","Créditos aprobados","Créditos excedentes"])], ignore_index=True)

    libre_cursadas = pd.DataFrame(columns=["Periodo Académico", "Código", "Asignatura", "Código", "Asignatura", "Nota", "#Créditos"])
    libre_cursadas.iloc[:,0] = cursadas_libre["Periodo"]
    libre_cursadas.iloc[:,1] = cursadas_libre["Cod_Asignatura"]
    libre_cursadas.iloc[:,2] = cursadas_libre["Nom_Asignatura"]
    libre_cursadas.iloc[:,3] = cursadas_libre["Cod_Asignatura"]
    libre_cursadas.iloc[:,4] = cursadas_libre["Nom_Asignatura"]
    libre_cursadas.iloc[:,5] = cursadas_libre["Calificacion"]
    libre_cursadas.iloc[:,6] = cursadas_libre["Creditos"]

    #Tabla de Resumen General
    resumen_general = pd.DataFrame(columns=["Créditos","Fund. Obl. B","Fund. Opt. O","Disc. Obl. C","Disc. Opt. T","Trabajo de Grado P","Libre Elección L", "Total"])
    resumen_general["Créditos"] = ["Exigidos","Convalidados/equivalentes","Pendientes","Excedentes"]

    resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Fund. Obl. B"] = agrupaciones[agrupaciones["Tipologia"]=="Fundamentación"]["Cant_Obligatorios"].sum()
    resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Fund. Opt. O"] = agrupaciones[agrupaciones["Tipologia"]=="Fundamentación"]["Cant_Optativos"].sum()
    resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Disc. Obl. C"] = agrupaciones[agrupaciones["Tipologia"]=="Disciplinar"]["Cant_Obligatorios"].sum()
    resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Disc. Opt. T"] = agrupaciones[agrupaciones["Tipologia"]=="Disciplinar"]["Cant_Optativos"].sum()
    resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Trabajo de Grado P"] = agrupaciones[agrupaciones["Tipologia"]=="Trabajo de grado"]["Cant_Optativos"].sum()
    resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Libre Elección L"] = agrupaciones[agrupaciones["Tipologia"]=="Libre Elección"]["Cant_Optativos"].sum()
    resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Total"] = resumen_general.loc[resumen_general["Créditos"] == "Exigidos", resumen_general.columns[1:]].sum(axis=1)

    resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Fund. Obl. B"] = cursadas_fundamentacion[cursadas_fundamentacion["Tipo"]=="B"]["Créditos"].astype(int).sum()
    resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Fund. Opt. O"] = cursadas_fundamentacion[cursadas_fundamentacion["Tipo"]=="O"]["Créditos"].astype(int).sum()
    resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Disc. Obl. C"] = cursadas_disciplinar[cursadas_disciplinar["Tipo"]=="C"]["Créditos"].astype(int).sum()
    resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Disc. Opt. T"] = cursadas_disciplinar[cursadas_disciplinar["Tipo"]=="T"]["Créditos"].astype(int).sum()
    # De acuerdo a la norma el trabajo de grado no es equivalible ni convalidable
    resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Trabajo de Grado P"] = 0
    resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Libre Elección L"] = libre_cursadas["#Créditos"].astype(int).sum()
    resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Total"] = resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", resumen_general.columns[1:]].sum(axis=1)


    resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Fund. Obl. B"] = tabla_pendientes_b["Créditos"].astype(int).sum()
    resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Disc. Obl. C"] = tabla_pendientes_c["Créditos"].astype(int).sum()
    resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Fund. Opt. O"] = pendientes_o["Créditos pendientes por cursar de la agrupación"].astype(int).sum()
    resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Disc. Opt. T"] = pendientes_t["Créditos pendientes por cursar de la agrupación"].astype(int).sum()
    resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Trabajo de Grado P"] = agrupaciones[agrupaciones["Tipologia"]=="Trabajo de grado"]["Cant_Optativos"].sum()
    resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Libre Elección L"] = max(0,resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Libre Elección L"].values[0] - resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Libre Elección L"].values[0])
    resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Total"] = resumen_general.loc[resumen_general["Créditos"] == "Pendientes", resumen_general.columns[1:]].sum(axis=1)

    # En las obligatorias nunca se obtienen créditos excedentes
    resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Fund. Obl. B"] = 0
    resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Disc. Obl. C"] = 0
    resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Trabajo de Grado P"] = 0
    resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Fund. Opt. O"] = creditos_contados[creditos_contados["Tipo"]=="Fundamentación"]["Créditos excedentes"].sum()
    resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Disc. Opt. T"] = creditos_contados[creditos_contados["Tipo"]=="Disciplinar"]["Créditos excedentes"].sum()
    resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Libre Elección L"] = creditos_contados[creditos_contados["Tipo"]=="Libre Elección"]["Créditos excedentes"].sum()
    resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Total"] = resumen_general.loc[resumen_general["Créditos"] == "Excedentes", resumen_general.columns[1:]].sum(axis=1)

    return resumen_general, cursadas_fundamentacion, cursadas_disciplinar, libre_cursadas, tabla_pendientes_b, pendientes_o, tabla_pendientes_disciplinar, pendientes_t

def Exportar_Estudio(resumen_general, cursadas_fundamentacion, cursadas_disciplinar, libre_cursadas, tabla_pendientes_b, pendientes_o, tabla_pendientes_disciplinar, pendientes_t):
    # Crear el documento Word
    doc = Document()

    # Configurar tamaño de papel A3 en orientación vertical (portrait)
    section = doc.sections[0]
    section.page_width = Mm(297)   # Ancho para A3 en portrait
    section.page_height = Mm(420)  # Alto para A3 en portrait

    # Función para establecer bordes negros en una tabla
    def set_table_border(table):
        tbl = table._tbl
        tblPr = tbl.tblPr
        if tblPr is None:
            tblPr = OxmlElement('w:tblPr')
            tbl.insert(0, tblPr)
        tblBorders = OxmlElement('w:tblBorders')
        for border in ('top', 'left', 'bottom', 'right', 'insideH', 'insideV'):
            element = OxmlElement(f'w:{border}')
            element.set(qn('w:val'), 'single')
            element.set(qn('w:sz'), '4')
            element.set(qn('w:space'), '0')
            element.set(qn('w:color'), '000000')
            tblBorders.append(element)
        tblPr.append(tblBorders)

    # Función que inserta un DataFrame como tabla en el documento
    def add_table_from_df(document, df, title=None):
        if title:
            document.add_heading(title, level=2)
        # Se crea la tabla con filas = cabecera + datos y columnas según el DataFrame
        table = document.add_table(rows=df.shape[0] + 1, cols=df.shape[1], style='Table Grid')

        # Agregar la fila de cabecera
        for j, col in enumerate(df.columns):
            table.cell(0, j).text = str(col)

        # Agregar las filas de datos
        for i in range(df.shape[0]):
            for j in range(df.shape[1]):
                table.cell(i+1, j).text = str(df.iloc[i, j])

        # Aseguramos que los bordes sean negros
        set_table_border(table)
        document.add_paragraph("")  # Línea en blanco para separar
        return table

    # Información inicial
    query_15 = """
    SELECT *
    FROM Planes_de_Estudio"""
    info_plan = pd.read_sql_query(query_15, Apuntador)

    doc.add_heading("1. Datos Generales", level=1)
    info_inicial = pd.DataFrame([
        ["Nombre", info_estudiante.loc[info_estudiante[0] == "Nombre:"][1].values[0]],
        ["DNI", info_estudiante.loc[info_estudiante[0] == "Documento:"][1].values[0]],
        ["Plan de estudios origen (1er plan):", info_estudiante.loc[info_estudiante[0] == "Plan:"][1].values[0]],
        ["Plan de estudios doble titulación (2° plan) - Sede",
        str(info_plan["Codigo_plan"].values[0]) + " - " + str(info_plan["Nombre_plan"].values[0])],
        ["Fecha de la solicitud", datetime.today().strftime("%d/%m/%Y")]
    ])
    add_table_from_df(doc, info_inicial)

    doc.add_heading("2. Información académica", level=1)
    reserva_cupo = "NO" if info_estudiante.loc[info_estudiante[0] == "Estado:"][1].values[0].split("\r")[0] == "Abierto" else "SI"
    porcentaje_avance = info_estudiante.loc[info_estudiante[0] == "Porcentaje de avance"][1].values[0]
    porcentaje_avance_num = float(porcentaje_avance.split("%")[0].replace(',', '.'))
    porcentaje_avance_40 = "SI" if porcentaje_avance_num >= 40 else "NO"
    papa = info_estudiante.loc[info_estudiante[0] == "P.A.P.A:"][1].values[0]
    creditos_dt = info_estudiante.loc[info_estudiante[0] == "Créditos de estudio doble titulación"][1].values[0]

    info_academica = pd.DataFrame([
        ["¿Tuvo calidad de estudiante en el 2° plan?", "NO"],
        ["¿Se encuentra matriculado o en reserva de cupo al momento de presentar la solicitud?", reserva_cupo],
        ["Porcentaje de avance en el plan de estudios origen", porcentaje_avance],
        ["¿El estudiante ha cursado el 40% de los créditos de su primer plan de estudios?", porcentaje_avance_40],
        ["P.A.P.A. en el primer plan de estudio", papa],
        ["Créditos estudio doble titulación (información reportada en el resumen de créditos del SIA)", creditos_dt]
    ])
    add_table_from_df(doc, info_academica)

    doc.add_heading("3. Resumen general de créditos del segundo plan de estudios:", level=1)
    add_table_from_df(doc, resumen_general)

    doc.add_heading("4. Cuadro de equivalencia y convalidaciones de asignaturas", level=1)

    doc.add_heading("Componente de FUNDAMENTACIÓN", level=2)
    cursadas_fundamentacion["Créditos_B"] = cursadas_fundamentacion.apply(lambda row: row["Créditos"] if row["Tipo"] == "B" else "", axis=1)
    cursadas_fundamentacion["Créditos_O"] = cursadas_fundamentacion.apply(lambda row: row["Créditos"] if row["Tipo"] == "O" else "", axis=1)
    cursadas_fundamentacion.drop(columns=["Tipo", "Créditos"], inplace=True)
    add_table_from_df(doc, cursadas_fundamentacion)

    doc.add_heading("Componente DISCIPLINAR", level=2)
    cursadas_disciplinar["Créditos_C"] = cursadas_disciplinar.apply(lambda row: row["Créditos"] if row["Tipo"] == "C" else "", axis=1)
    cursadas_disciplinar["Créditos_T"] = cursadas_disciplinar.apply(lambda row: row["Créditos"] if row["Tipo"] == "T" else "", axis=1)
    cursadas_disciplinar.drop(columns=["Tipo", "Créditos"], inplace=True)
    add_table_from_df(doc, cursadas_disciplinar)

    doc.add_heading("Componente de LIBRE ELECCIÓN", level=2)
    add_table_from_df(doc, libre_cursadas)

    doc.add_heading("5. Asignaturas pendientes por cursar en el segundo plan de estudios", level=1)
    doc.add_paragraph("Pendientes en Fundamentación - Obligatoria B")
    add_table_from_df(doc, tabla_pendientes_b)
    doc.add_paragraph("Pendientes en Fundamentación - Optativa O")
    add_table_from_df(doc, pendientes_o)
    doc.add_paragraph("Pendientes en Disciplinar - Obligatoria C y P")
    add_table_from_df(doc, tabla_pendientes_disciplinar)
    doc.add_paragraph("Pendientes en Disciplinar - Optativa T")
    add_table_from_df(doc, pendientes_t)

    add_table_from_df(doc,pd.DataFrame(columns=["Componente de Libre Elección (L) (Créditos pendientes)",resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Libre Elección L"].values[0]]))

    doc.add_heading("6. El Consejo de Facultad Recomienda / No recomienda", level=1)
    recomienda = None
    razon = ""
    if porcentaje_avance_num < 40:
        recomienda = False
        razon = "No dispone de un porcentaje de avance mayor al 40 en el primer plan de estudios."
    elif float(papa) >= 4.3:
        recomienda = True
        if int(creditos_dt) >= int(resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Total"].values[0]):
          razon = "Dispone de los créditos suficientes y además presenta un PAPA mayor o igual a 4.3"
        else:
          razon = ("Aunque no dispone de los créditos suficientes para cursar las asignaturas pendientes de "
                "aprobación en el segundo plan, presenta un PAPA mayor o igual a 4,3.")
    elif int(resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Total"].values[0]) > int(creditos_dt):
        recomienda = False
        razon = "No dispone de los créditos suficientes para cursar las asignaturas pendientes de aprobación en el segundo plan."
    else:
        recomienda = True
        razon = ("Dispone de los créditos suficientes para cursar las asignaturas pendientes de aprobación "
                "en el segundo plan de estudios.")

    doc.add_paragraph("Recomendación: " + ("Recomienda" if recomienda else "No recomienda"))
    doc.add_paragraph(razon)

    # Guardar el documento Word
    nombre_estudiante = info_estudiante.loc[info_estudiante[0] == "Nombre:"][1].values[0]
    doc.save(f"{nombre_estudiante}_estudio_dt.docx")

info_estudiante, asignaturas_estudiante = Obtener_Informacion_PDF("HistoriaAcadem2.pdf")
lista_candidatas = Generar_Lista_Candidatas(asignaturas_estudiante)
resumen_general, cursadas_fundamentacion, cursadas_disciplinar, libre_cursadas, tabla_pendientes_b, pendientes_o, tabla_pendientes_disciplinar, pendientes_t = Generar_Estudio(info_estudiante,lista_candidatas)
Exportar_Estudio(resumen_general, cursadas_fundamentacion, cursadas_disciplinar, libre_cursadas, tabla_pendientes_b, pendientes_o, tabla_pendientes_disciplinar, pendientes_t)


Feb 20, 2025 7:11:54 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

